In [ ]:
import selenium
from selenium import webdriver as wb
import pandas as pd
import numpy as np
import datetime
import time
from datetime import date


In [ ]:
get_ipython().run_line_magic('cd', 'C:/Users/rasla/OneDrive/Desktop/sel.py/chromedriver.exe')
driver = wb.Chrome("C:/Users/rasla/OneDrive/Desktop/sel.py/chromedriver.exe")

In [ ]:
month = [11,11,11,11,11,11,11]
month = [str(x).zfill(2) for x in month]
day = [17,18,19,20,21,22,23,24,25]
day = [str(x).zfill(2) for x in day]
year = [2022,2022,2022,2022,2022,2022,2022]
year = [str(x).zfill(4) for x in year]

In [ ]:
Flight_Prices = pd.DataFrame()

for a,b,c in zip(day,month,year):
    driver.get("https://www.makemytrip.com/flight/search?tripType=O&itinerary=ATQ-CCJ-{}/{}/{}&paxType=A-1_C-0_I-0&cabinClass=E&sTime=1597828876664&forwardFlowRequired=true".format(a,b,c))
    time.sleep(15)
    
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True
    
    time.sleep(60)
    FlightName_elements = driver.find_elements_by_xpath("//div[@class='pull-left airways-info-sect']")
    FlightName_elements = [x.text for x in FlightName_elements]
    FlightName = [x.split('\n')[0] for x in FlightName_elements]
    FlightName = pd.Series(FlightName)
    
    Price_elements = driver.find_elements_by_xpath("//span[@class='actual-price']")
    Price = [x.text for x in Price_elements]
    Price = [i for i in Price if i]
    Price = pd.Series(Price)
    
    Fromcity_elements = driver.find_elements_by_xpath("//p[@class='dept-city']")
    Fromcity = [x.text for x in Fromcity_elements]
    Fromcity = pd.Series(Fromcity)
    
    Tocity_elements = driver.find_elements_by_xpath("//p[@class='arrival-city']")
    Tocity = [x.text for x in Tocity_elements]
    Tocity = pd.Series(Tocity)
    
    Duration_elements = driver.find_elements_by_xpath("//p[@class='fli-duration']")
    Duration = [x.text for x in Duration_elements]
    Duration = pd.Series(Duration)
    
    Deptime_elements = driver.find_elements_by_xpath("//div[@class='dept-time']")
    Deptime = [x.text for x in Deptime_elements]
    Deptime = pd.Series(Deptime)
    
    Arrtime_elements = driver.find_elements_by_xpath("//p[@class='reaching-time append_bottom3']")
    Arrtime = [x.text for x in Arrtime_elements]
    Arrtime = [x.split("+", 1)[0] for x in Arrtime]
    Arrtime = pd.Series(Arrtime)
    
    Date_elements = driver.find_elements_by_xpath("//div[@class='item blue_active']")
    Date_elements = [x.text for x in Date_elements]
    x = [x.split(',', 1)[1] for x in Date_elements]
    Date = [i.split('\n', 1)[0] for i in x]
    Date = pd.Series(Date)
            
    df = pd.DataFrame({'Date':Date,"Airline":FlightName,"From City":Fromcity, "To City":Tocity, "Departure Time":Deptime,"Arrival Time":Arrtime,"Flight Duration":Duration,"Price":Price})
    
    Flight_Prices = Flight_Prices.append(df) 
        
Flight_Prices[Flight_Prices.Date==""] = np.NaN
Flight_Prices.Date = Flight_Prices.Date.fillna(method='ffill')
Flight_Prices.Price = Flight_Prices.Price.str.replace(",","").str.extract('(\d+)')
Flight_Prices


In [ ]:
Flight_Prices = Flight_Prices[Flight_Prices['Price'].notna()]
Flight_Prices['Price'] = pd.to_numeric(Flight_Prices['Price'])

In [ ]:
Flight_Prices = Flight_Prices[(Flight_Prices['Departure Time']>='15:00') & (Flight_Prices['Departure Time']<= '18:00')]
Flight_Prices

In [ ]:
Flight_Prices = Flight_Prices.loc[Flight_Prices.groupby('Date')['Price'].idxmin()]
Flight_Prices = Flight_Prices.drop_duplicates('Date')
Flight_Prices